In [16]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [17]:
path = "/home/kotik/dataset/letter-recognition.data"
data = pd.read_csv(path, header= None)

In [18]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10


In [19]:
data.shape

(20000, 17)

In [20]:
labels = data[0]
input_data = data.drop(0, axis=1)

In [21]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.losses import *
from keras.metrics import *
from keras.callbacks import *

In [22]:
data.describe()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
count,20000.000000,20000.000000,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000
mean,4.023550,7.035500,5.121850,5.37245,3.505850,6.897600,7.500450,4.628600,5.178650,8.282050,6.45400,7.929000,3.046100,8.338850,3.691750,7.80120
std,1.913212,3.304555,2.014573,2.26139,2.190458,2.026035,2.325354,2.699968,2.380823,2.488475,2.63107,2.080619,2.332541,1.546722,2.567073,1.61747
min,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,3.000000,5.000000,4.000000,4.00000,2.000000,6.000000,6.000000,3.000000,4.000000,7.000000,5.00000,7.000000,1.000000,8.000000,2.000000,7.00000
50%,4.000000,7.000000,5.000000,6.00000,3.000000,7.000000,7.000000,4.000000,5.000000,8.000000,6.00000,8.000000,3.000000,8.000000,3.000000,8.00000
75%,5.000000,9.000000,6.000000,7.00000,5.000000,8.000000,9.000000,6.000000,7.000000,10.000000,8.00000,9.000000,4.000000,9.000000,5.000000,9.00000
max,15.000000,15.000000,15.000000,15.00000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.00000,15.000000,15.000000,15.000000,15.000000,15.00000


In [23]:
labels.unique()

array(['T', 'I', 'D', 'N', 'G', 'S', 'B', 'A', 'J', 'M', 'X', 'O', 'R',
       'F', 'C', 'H', 'W', 'L', 'P', 'E', 'V', 'Y', 'Q', 'U', 'K', 'Z'],
      dtype=object)

In [24]:
data.groupby(0).size()

0
A    789
B    766
C    736
D    805
E    768
F    775
G    773
H    734
I    755
J    747
K    739
L    761
M    792
N    783
O    753
P    803
Q    783
R    758
S    748
T    796
U    813
V    764
W    752
X    787
Y    786
Z    734
dtype: int64

In [25]:
#преобразование данных
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

labels_num = LabelEncoder().fit_transform(labels)
labels_output = to_categorical(labels_num)
input_train, input_test, labels_out_train, labels_out_test = train_test_split(input_data, labels_output, test_size=0.3)

In [26]:
n_input = input_data.shape[1]
n1 = 128
n2 = 64
n_output = len(labels.unique())

In [27]:
# model = Sequential()  один способ задания NN
# model.add(Input())
# model.add(Dense())
# model.add(.....)

In [28]:
inp = Input(shape=(n_input,)) # (batch_size, n_input)
dense1 = Dense(n1, activation = 'relu')(inp) # после какого слоя идет, выход какого слоя использует
# activ = LeakyReLu()(dense1) # можно добавлять как слои
# dropout = Dropout()(dense1)
# bn = BatchNormalization()(dense1)
dense2 = Dense(n1, activation = 'relu')(dense1)
output = Dense(n_output, activation = 'softmax')(dense2)
model = Model(inputs=inp, outputs=output)

In [53]:
model.compile(RMSprop(lr=0.001), loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

filepath = "/home/kotik/code/Data_Analysis/letter_recognition_checkpoints/weights-{val_categorical_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, save_best_only=True, monitor='val_categorical_accuracy', mode='max')
#сохрани checkpoint, если он лучше всех предыдущих по заданной метрике


In [55]:
model.fit(input_train, labels_out_train, batch_size=100, verbose=2, epochs=200, # verbose - говорливая тренировка
          validation_data=(input_test, labels_out_test), callbacks = [checkpoint]) 



Train on 14000 samples, validate on 6000 samples
Epoch 1/200
 - 1s - loss: 0.0104 - categorical_accuracy: 0.9971 - val_loss: 0.3719 - val_categorical_accuracy: 0.9498
Epoch 2/200
 - 1s - loss: 0.0070 - categorical_accuracy: 0.9981 - val_loss: 0.3878 - val_categorical_accuracy: 0.9490
Epoch 3/200
 - 1s - loss: 0.0104 - categorical_accuracy: 0.9969 - val_loss: 0.3324 - val_categorical_accuracy: 0.9522
Epoch 4/200
 - 1s - loss: 0.0083 - categorical_accuracy: 0.9973 - val_loss: 0.3147 - val_categorical_accuracy: 0.9528
Epoch 5/200
 - 1s - loss: 0.0117 - categorical_accuracy: 0.9970 - val_loss: 0.3190 - val_categorical_accuracy: 0.9570
Epoch 6/200
 - 1s - loss: 0.0114 - categorical_accuracy: 0.9970 - val_loss: 0.3306 - val_categorical_accuracy: 0.9547
Epoch 7/200
 - 1s - loss: 0.0087 - categorical_accuracy: 0.9972 - val_loss: 0.3475 - val_categorical_accuracy: 0.9533
Epoch 8/200
 - 1s - loss: 0.0113 - categorical_accuracy: 0.9971 - val_loss: 0.3960 - val_categorical_accuracy: 0.9448
Epoch 9

KeyboardInterrupt: 

In [ ]:
# нормализовать вход от 0 до 1, хороший тон, можно улучшить метрику

In [ ]:
# То же в PyTorch

In [56]:
import torch
import torch.nn as nn

ImportError: /home/kotik/anaconda3/envs/naive_bayes/lib/python3.7/site-packages/torch/lib/libtorch.so.1: undefined symbol: nvrtcGetProgramLogSize

In [ ]:
#преобразование данных
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

labels_num = LabelEncoder().fit_transform(labels)
input_train, input_test, labels_out_train, labels_out_test = train_test_split(input_data, labels_num, test_size=0.3)

In [ ]:
x_train - tensor(input_train)
y_train = tensor(labels_out_train)
x_test = tensor(input_test)
y_test = tensor(labels_out_test)
dataset_train = torch.utils.data.TensorDataset(x_train,y_train)
dataset_test = torch.utils.data.TensorDataset(x_test,y_test)
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=100,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=100,shuffle=True)


In [ ]:
model = nn.Sequential(nn.Linear(n_input, n1),
                     nn.SELU(),
                     nn.Linear(n1, n2),
                     nn.SELU(),
                     nn.Linear(n2, n_output),
                     nn.Softmax(dim=1))

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters.lr = 0.1)

In [ ]:
for epoch in range(200):
    for i, data_batch in enumerate(train_loader, 0):
        optimizer.zero_grad()
        x, y = data_batch
        y_pred = model(x.float())
        
        loss = criterion(y_pred, y)
        loss.bachward()
        
        optimizer.step()
    print('epoch:', epoch+1, 'loss', loss.item())
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            x, y = data
            outputs = model(x.float())
            _,predicted = torch.max(outputs.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
        print("Accurancy:" predicted/total)
    
    